In [19]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import risk_kit as rk
import matplotlib.pyplot as plt


def show_cppi(n_scenarios=1000, mu=0.07, sigma=0.15, m=3, floor=0, riskfree_rate=0.03, y_max=100):
    start = 100
    sim_rets = rk.gbm(n_scenarios=n_scenarios, mu=mu, sigma=sigma, prices=False, steps_per_year=12)
    risky_r = pd.DataFrame(sim_rets)
    btr = rk.run_cppi(risky_r=pd.DataFrame(risky_r), riskfree_rate=riskfree_rate, m=m, start=start, floor=floor)
    wealth = btr["Wealth"]
    
    y_max = wealth.values.max() * y_max/100
    terminal_wealth = wealth.iloc[-1]
    
    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    failure_mask = np.less(terminal_wealth, start*floor)
    n_failures = failure_mask.sum()
    p_fail = n_failures/n_scenarios
    
    e_shortfall = np.dot(terminal_wealth-start*floor, failure_mask)/n_failures if n_failures > 0 else 0
    
    fig, (wealth_ax, hist_ax)=plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={"width_ratios":[3,2]}, figsize=(24,9))
    plt.subplots_adjust(wspace=0.0)
    
    wealth.plot(ax=wealth_ax, legend=False, color="indianred", alpha=0.3)
    wealth_ax.axhline(y=start, ls=":", color="black")
    wealth_ax.axhline(y=start*floor, ls="--", color="red")
    wealth_ax.set_ylim(top=y_max)
    
    terminal_wealth.plot.hist(ax=hist_ax, bins=50, ec="w", fc="indianred", orientation="horizontal")
    hist_ax.axhline(y=start, ls=":", color="black")
    hist_ax.axhline(y=tw_mean, ls=":", color="blue")
    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(0.7, 0.9), xycoords="axes fraction", fontsize=20)
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy=(0.7, 0.84), xycoords="axes fraction", fontsize=20)
    if floor > 0.01:
        hist_ax.axhline(y=start*floor, ls=":", color="red", linewidth=3)
        hist_ax.annotate(f"Violations: {n_failures} ({p_fail*100:2.2f}%)\nE(shortfall)=${e_shortfall:2.2f}", xy=(0.7, 0.7), xycoords="axes fraction", fontsize=20)

    
cppi_controls = widgets.interactive(show_cppi,
                                   n_scenarios=widgets.IntSlider(min=1, max=1000, step=5, value=50),
                                   mu=(0, +0.2, 0.01),
                                   sigma=(0, 0.5, 0.05),
                                   floor=(0, 2, 0.1),
                                   m=(1, 5, 0.5),
                                   riskfree_rate=(0, 0.05, 0.01),
                                   y_max=widgets.IntSlider(min=0, max=100, step=1, value=100, description="Zoom Y Axis"))

display(cppi_controls)


interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…